In [115]:
import tensorflow as tf
model = tf.saved_model.load('../models/efficientdet_d1_coco17_tpu-32/saved_model')


In [162]:
import cv2

def load_and_preprocess_image(path):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Resize if necessary
    # image = cv2.resize(image, (desired_width, desired_height))

    # Convert to uint8
    image = tf.convert_to_tensor(image, dtype=tf.uint8)

    # Add batch dimension
    image = tf.expand_dims(image, 0)
    return image

image = load_and_preprocess_image('latest.jpg')


In [163]:
detections = model(image)

In [164]:
class_names = {}
with open('../models/coco_labels.txt', 'r') as file:
    for i, line in enumerate(file.readlines(), start=1):
        class_names[i] = line.strip()

In [165]:
# Extract bounding boxes from the model's output
boxes = detections['detection_boxes'].numpy()
classes = detections['detection_classes'].numpy()
scores = detections['detection_scores'].numpy()

# Load the original image
original_image = cv2.imread('latest.jpg')

# Make sure the image was loaded correctly
if original_image is None:
    raise ValueError("Image not found or path is incorrect")

# Original image dimensions
height, width, _ = original_image.shape  # original_image is your original image

for i in range(boxes.shape[1]):
    if scores[0, i] > 0.5:  # Score threshold
        box = boxes[0, i]
        class_id = int(classes[0, i])

        # Scale box to original image dimensions
        ymin, xmin, ymax, xmax = box
        xmin = int(xmin * width)
        xmax = int(xmax * width)
        ymin = int(ymin * height)
        ymax = int(ymax * height)

        # Draw the bounding box
        cv2.rectangle(original_image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

        # Draw label
        label = f'{class_names.get(class_id, "N/A")} : {int(scores[0, i] * 100)}%'
        cv2.putText(original_image, label, (xmin, ymin-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


In [166]:
cv2.imshow('Object Detection', original_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [182]:
detected_objects = []

for i in range(boxes.shape[1]):
    score = scores[0, i]
    if score > 0.5:
        class_id = int(classes[0, i])
        label = class_names.get(class_id, "N/A")
        if label in ['2  car', '86  scissors']:
            ymin, xmin, ymax, xmax = boxes[0, i]
            xmin = int(xmin * width)
            xmax = int(xmax * width)
            ymin = int(ymin * height)
            ymax = int(ymax * height)

            detected_object = {
                'xmin': xmin,
                'ymin': ymin,
                'xmax': xmax,
                'ymax': ymax
            }

            # Add the object to the list
            detected_objects.append(detected_object)


print(f"Filtered detections: {len(detected_objects)}")
print(detected_objects)


Filtered detections: 6
[{'xmin': 262, 'ymin': 335, 'xmax': 516, 'ymax': 443}, {'xmin': 267, 'ymin': 128, 'xmax': 502, 'ymax': 243}, {'xmin': 238, 'ymin': 532, 'xmax': 507, 'ymax': 661}, {'xmin': 833, 'ymin': 561, 'xmax': 1104, 'ymax': 686}, {'xmin': 821, 'ymin': 328, 'xmax': 1071, 'ymax': 448}, {'xmin': 810, 'ymin': 146, 'xmax': 1038, 'ymax': 253}]


[]
